## Tourist Site Data

Instructions:

1. Extract tourist site data (Note: at least 25 different tourist sites)
    a. Tourist Site Name
    b. Classification (e.g. museum, park, zoo, etc.)
    c. Location (Note: Barangay, City/Municipality, Province)
    d. Number of visitors per year (Note: in thousands)
    e. Entrance fee (Note: Peso)
    f. Other data (extract more regarding tourist sites that can be used by tourists)
    g. customer reviews
2. Store collected data to a csv file. Filename format:  Tourist_Site.csv
3. Please upload your Jupyter notebook file. Filename format:
    Extract_Tourist_Site_Source.ipynb
    

Source: Travel Advisor - website

In [1]:
#import important libraries

import numpy as np
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import urllib.request
import re #regular expression
import lxml
from time import sleep
from random import randint

In [2]:
#get data from Trip Advisor

url = 'https://www.tripadvisor.com.ph/Attractions-g298449-Activities-a_allAttractions.true-Metro_Manila_Luzon.html'

#connect to website
page = urllib.request.urlopen(url) 

#convert to beautiful soup object
soup = BeautifulSoup(page, 'lxml')
print(soup)

<!DOCTYPE html>
<html lang="en"><head><meta content="text/html; charset=utf-8" http-equiv="content-type"/><meta content="en" http-equiv="content-language"/><link href="https://static.tacdn.com/favicon.ico?v2" id="favicon" rel="icon" type="image/x-icon"/><link color="#000000" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" rel="mask-icon" sizes="any"/><meta content="#34e0a1" name="theme-color"/><meta content="width=device-width, initial-scale=1.0, viewport-fit=cover" name="viewport"/><meta content="telephone=no" name="format-detection"/><meta content="TripAdvisor" property="al:ios:app_name"/><meta content="284876795" property="al:ios:app_store_id"/><meta content="284876795" name="twitter:app:id:ipad" property="twitter:app:id:ipad"/><meta content="284876795" name="twitter:app:id:iphone" property="twitter:app:id:iphone"/><meta content="tripadvisor://www.tripadvisor.com.ph/Attractions-g298449-Activities-a_allAttractions.true-Metro_Manila_Luzon.html?m=3376

In [3]:
#create a list where the link of each tourist site from TripAdvisor will be stored

links_mm = []

In [4]:
#find all links of each tourist site from Trip Advisor and store to the links_mm list

for review in soup.findAll('a',{'target': '_blank'}): #'_7c6GgQ6n _37QDe3gr WullykOU _3WoyIIcL'}):
    a = review['href']
    a = 'https://www.tripadvisor.in'+ a
    a = a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
    links_mm.append(a)


In [5]:
#check content of links_mm
links_mm

['https://www.tripadvisor.in/Attraction_Review-g298573-d310887-Reviews-or{}-National_Museum-Manila_Metro_Manila_Luzon.html',
 'https://www.tripadvisor.in/Attraction_Review-g298573-d310887-Reviews-or{}-National_Museum-Manila_Metro_Manila_Luzon.html',
 'https://www.tripadvisor.in/Attraction_Review-g298573-d310887-Reviews-or{}-National_Museum-Manila_Metro_Manila_Luzon.html',
 'https://www.tripadvisor.in/Attraction_Review-g298573-d310887-Reviews-or{}-National_Museum-Manila_Metro_Manila_Luzon.html#REVIEWS',
 'https:-or{}//www.tripadvisor.in/Profile/vivas_margarida',
 'https:-or{}//www.tripadvisor.in/Profile/vivas_margarida',
 'https://www.tripadvisor.in/ShowUserReviews-or{}-g298573-d310887-r744401812-National_Museum-Manila_Metro_Manila_Luzon.html',
 'https://www.tripadvisor.in/Attraction_Review-g298450-d2648995-Reviews-or{}-Greenbelt_Mall-Makati_Metro_Manila_Luzon.html',
 'https://www.tripadvisor.in/Attraction_Review-g298450-d2648995-Reviews-or{}-Greenbelt_Mall-Makati_Metro_Manila_Luzon.htm

In [6]:
#The initial list contains data quality issues. To get the links of the tourist sites from Trip Advisor, we filter the contents
#of links_mm by getting those rows with prefix 'https://www.tripadvisor.in/Attraction_Review-' and suffix '#REVIEWS'

#create links_mm_2 that will contain the final list of tourist site links
links_mm_2 = []

#filter links_mm content and store to links_mm_2
for i in range(0,len(links_mm)):
    if ('https://www.tripadvisor.in/Attraction_Review-' in links_mm[i]) & ('#REVIEWS' in links_mm[i]):
        links_mm_2.append(links_mm[i])

#check contents of links_mm_2
links_mm_2


['https://www.tripadvisor.in/Attraction_Review-g298573-d310887-Reviews-or{}-National_Museum-Manila_Metro_Manila_Luzon.html#REVIEWS',
 'https://www.tripadvisor.in/Attraction_Review-g298450-d2648995-Reviews-or{}-Greenbelt_Mall-Makati_Metro_Manila_Luzon.html#REVIEWS',
 'https://www.tripadvisor.in/Attraction_Review-g298574-d7396573-Reviews-or{}-Art_in_Island-Quezon_City_Metro_Manila_Luzon.html#REVIEWS',
 'https://www.tripadvisor.in/Attraction_Review-g298573-d586732-Reviews-or{}-Fort_Santiago-Manila_Metro_Manila_Luzon.html#REVIEWS',
 'https://www.tripadvisor.in/Attraction_Review-g298452-d1563522-Reviews-or{}-Resorts_World_Manila-Pasay_Metro_Manila_Luzon.html#REVIEWS',
 'https://www.tripadvisor.in/Attraction_Review-g1758900-d4211937-Reviews-or{}-Bonifacio_Global_City-Taguig_City_Metro_Manila_Luzon.html#REVIEWS',
 'https://www.tripadvisor.in/Attraction_Review-g298573-d548076-Reviews-or{}-Intramuros-Manila_Metro_Manila_Luzon.html#REVIEWS',
 'https://www.tripadvisor.in/Attraction_Review-g298452

In [7]:
#create dataframe that will contain the tourist reviews per tourist site

columns_1 = ['Tourist Site', 'review_1_title', 'review_1',
    'review_2_title', 'review_2', 'review_3_title', 'review_3', 'review_4_title', 'review_4',
    'review_5_title', 'review_5', 'review_6_title', 'review_6', 'review_7_title', 'review_7',
    'review_8_title', 'review_8', 'review_9_title', 'review_9', 'review_10_title', 'review_10',
    'review_11_title', 'review_11', 'review_12_title', 'review_12', 'review_13_title', 'review_13',
     'review_14_title', 'review_14', 'review_15_title', 'review_15', 'review_16_title', 'review_16',
     'review_17_title', 'review_17', 'review_18_title', 'review_18', 'review_19_title', 'review_19',
     'review_20_title', 'review_20']

df_review = pd.DataFrame(columns = columns_1, index = range(0, len(links_mm_2)+1))

#check dataframe structure
df_review

,Tourist Site,review_1_title,review_1,review_2_title,review_2,review_3_title,review_3,review_4_title,review_4,review_5_title,...,review_16_title,review_16,review_17_title,review_17,review_18_title,review_18,review_19_title,review_19,review_20_title,review_20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#This section extracts the tourists' reviews on each tourist site. The extracted reviews from each tourist site were stored in
#the df_reviews


d = ['','10'] #'' and 10 will be added to each tourist site link. These pertain to the page 1 and 2 of the tourist site webpage
j = 0
l = 0

for link in links_mm_2:
    reviews = [] #will temporarily contain the reviews data
    reviews_2 = [] #will temporarily contain the cleaned reviews data

    for i in range(1,len(d)+1):
        link2 = link.replace("{}",str(d[i-1])) #replace {} with '' or 10 to extract the reviews data from page 1 and 2 of the website
        soup2 = urllib.request.urlopen(link2)
        sleep(randint(1,5))
        bsobj2 = BeautifulSoup(soup2,'lxml')
        if i == 1:
            #extract the headers, which contains the name of the tourist site and append to reviews
            for s in bsobj2.find_all('h1', {'class':'DrjyGw-P _1SRa-qNz qf3QTY0F'}):
                reviews.append(s.getText().split('\n')[0])
        
        #extract reviews
        for w in bsobj2.findAll('span','_2tsgCuqy'):
                reviews.append(w.getText().split('\n')[0])
    
    #filter the values extracted from the website. Some of the data extracted were not tourist reviews
    for m in range(0,len(reviews)):
        if 'from ₹' not in reviews[m]:
            reviews_2.append(reviews[m]) #store filtered data to reviews_2

    #store reviews_2 to df_review
    for k in range(0,len(reviews_2)):
        if k < len(columns_1):
            df_review.iloc[l,k] = reviews_2[k]
            
    
    if l < len(links_mm_2)-1:
        l = l +1

In [9]:
#check content

df_review

,Tourist Site,review_1_title,review_1,review_2_title,review_2,review_3_title,review_3,review_4_title,review_4,review_5_title,...,review_16_title,review_16,review_17_title,review_17,review_18_title,review_18,review_19_title,review_19,review_20_title,review_20
0,National Museum,Love the paintings,We really loved the paintings at the National ...,Great History,"Upon entering, the first major painting was th...",It is ok,"Structure is ok but contentwise, nothing much....",Very informative and entertaining,This is one of the 4 part of National Museum i...,Great place to visit.,...,Truly amazing Filipino art,One of the underrated repositories of art in A...,Allot one whole day for the trip,Allot one whole day for you and your family to...,A nice museum to see and cool off in.,The PI is not a rich nation and lacks the dept...,Underrated Museum,This museum is located in the heart of the cou...,It's ok especially for the price (free!),Many exhibits covering different regions of Ph...
1,Greenbelt Mall,Nice mall,Nice mall for good walk and shopping! You wil...,Love the mall been there nov 2019 April 2019,"Restaurant and coffee shops are everywhere, br...",Typical high end mall,"This is a high end mall, not that different fr...",Great atmosphere,Such a maze of shops and restaurants. Lovely t...,Good shopping options,...,great locatio,nice mall favoritr one more to offrr likr rest...,Great place in the center of Manila,Great place for shopping and dining out Lots o...,One of our favorite malls in the Philippines,Greenbelt mall is very large with 5 malls in o...,Gigantic mall,Gosh this place was overwhelming at times but ...,"Greenbelt, place to visit",Greenbelt is a nice a place to to shop and eat...
2,Art in Island,Exciting cultural Experience,it's a one-of-a-kind showcase of art. The 3-d...,amazing place to increase your imagination,Explore your creativity on all the pictures yo...,"Do once, take all the photos you can",This was the second time I've gone to art in i...,Excellent!,"Amazing place, worth the visit! Loads of fun.....",One-Thousand-Photo-per-Hour Challenge,...,Art in Island,A fun and memorable experience especially for ...,A fun place to visit,This place is a lot of fun to visit with some ...,"Fun, family-friendly venue for photo-ops or In...",Be part of the art - take photos from prescrib...,A PLACE TO ENJOY!,"Me, my mom and my kids enjoyed our visit here....","A must-go for photo""holics""",Best time to go is when you have already eaten...
3,Fort Santiago,The Spanish heritage,You can see the Spanish glory in the beautiful...,Structurally beautiful,"As Christians, this place boost My faith, feed...",Worth the trip,Pretty park in the old city. It was nice to w...,Okay place.,"Old Fort, most of it crumbling. Some parts clo...",Good historical place,...,Good Photo Opportunities!,"Beautiful place, so clean and many scenic area...","Another Fort? Yes, Do It!",Fort Santiago is a treasure in the middle of t...,Fort santiago,Place is well maintained. I could see that the...,Experience the history of Manila!,This one will make you see the history of Riza...,Part of the old town,"When you go to Intramuros, the old town of Man..."
4,Resorts World Manila,Cool mall,Didn't expect that the casino had a small mall...,Ambient Mall,You can find luxury goods here... enjoyed the ...,Impressive,We were impressed at the huge complex built he...,Beautiful Mall,The mall is like a city within a city.....lots...,Traditional Casino Experience,...,Playing in Casino,This is a highly recommended hotel especially ...,Great Casino and area,"Enjoyed it here, very up market, great casino ...",Feast,When me and my friend go to resorts world mani...,Resorts world,Located in Manila not far from Terminal 3 airp...,Fun casino fun people good food,I enjoyed playing here even if 8 didn't win an...
5,Bonifacio Global City,Walk for Leisure.,"A great place to hang-out. Clean, modern and s...",Skyscrapers and concrete,"Nothing to to with Pilipino spirit, but defini..."

In [10]:
#create dataframe that will contain the reviewers data

columns_2 = ['Tourist Site','reviewer_1','reviewer_2','reviewer_3','reviewer_4','reviewer_5','reviewer_6','reviewer_7',
             'reviewer_8','reviewer_9','reviewer_10','reviewer_11','reviewer_12','reviewer_13','reviewer_14','reviewer_15',
             'reviewer_16','reviewer_17','reviewer_18','reviewer_19','reviewer_20']

df_reviewer = pd.DataFrame(columns = columns_2, index = range(0,len(links_mm_2)))

#check structure
df_reviewer

,Tourist Site,reviewer_1,reviewer_2,reviewer_3,reviewer_4,reviewer_5,reviewer_6,reviewer_7,reviewer_8,reviewer_9,...,reviewer_11,reviewer_12,reviewer_13,reviewer_14,reviewer_15,reviewer_16,reviewer_17,reviewer_18,reviewer_19,reviewer_20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#This section extracts the reviewers data from each tourist site webpage and store to df_reviewer dataframe


reviewers_2 = [] #will temporarily contain the cleaned reviewers data

l = 0

#using for loop, extract reviewers data from each tourist site webpage

for link in links_mm_2:
    for i in range(1,len(d)+1):
        reviewers = [] #will temporarily contain the cleaned reviewers data
        link2 = link.replace("{}",str(d[i-1])) #replace {} with contents of d to extract data from page 1 to page 2
        soup2 = urllib.request.urlopen(link2)
        sleep(randint(1,5))
        bsobj2 = BeautifulSoup(soup2,'lxml')
        if i == 1:
            #extract the headers, which contains the name of the tourist site and append to reviewers
            for s in bsobj2.find_all('h1', {'class':'DrjyGw-P _1SRa-qNz qf3QTY0F'}):
                reviewers.append(s.getText().split('\n')[0])
        
        #extract reviewers
        for w in bsobj2.findAll('span',{'class':'DrjyGw-P _1SRa-qNz NGv7A1lw _2yS548m8 _2cnjB3re _1TAWSgm1 _1Z1zA2gh _2-K8UW3T _2AAjjcx8'}):
            reviewers.append(w.getText().split('\n')[0])
        
        #filter data and store to reviewers_2
        for j in range(0,len(reviewers)):
            if j <=11:
                reviewers_2.append(reviewers[j])
                
    #store content of reviewers_2 to df_reviewer
    for k in range(0,len(reviewers_2)):
        if k < len(columns_2):
            df_reviewer.iloc[l,k] = reviewers_2[k]
    reviewers_2 = []
    if l <= len(links_mm_2):
        l = l + 1

In [12]:
#check content of df_reviewer
df_reviewer

,Tourist Site,reviewer_1,reviewer_2,reviewer_3,reviewer_4,reviewer_5,reviewer_6,reviewer_7,reviewer_8,reviewer_9,...,reviewer_11,reviewer_12,reviewer_13,reviewer_14,reviewer_15,reviewer_16,reviewer_17,reviewer_18,reviewer_19,reviewer_20
0,National Museum,Hellen Mai,audreyb120,Citygal12456,Ron,Sarge57,elxar,angelicious0327,Las Buganvillas,Hmcc85,...,Joy T,Elaine F,Siegfred A,Indy27,Panamalor,Steven T,Choo-Beng G,Jennifer G,gea4957,CounselorEK
1,Greenbelt Mall,grace virlouvet,Garland V,KGB777,Mel LM,Espen V,Hammer13,Jy Go,eenski,Alan M,...,Footprints263173,Nello G,MinsMarket,Yvet D,Casey,Elaine F,Eugene,Waleed AlWafi,International_Traveler,Timetravelbug
2,Art in Island,Jane V,milo_dapilos,84Cerberus84,Jodi J,Daring Darling,wena_anew,Mharl Cua,Saul_True,Tina L,...,remalyn m,dunn,regidorvillegas,brianandtts,detceleb,Cassandra Yu,Jase,Darren B,JAASM,zheeby2872
3,Fort Santiago,Elena Ciocan,Marimar A,crlyn85,Shafiq H,Citygal12456,Culture625967,coyg2019,KGB777,Sarge57,...,christian c,Hammer13,Pat r,Luz C,Makoto,Bernadine S,Mambo305,Alan L,Nico Samneil Sy Salado,dongjerms
4,Resorts World Manila,Mr Prinz,Fred,MountaineerLu,Tenkid,Fr. Bruce,ed M,Hammer13,Darryn,Makoto,...,Wanderer785704,Yvet D,Ryan S,John C,Lydia Imperial M,Al ali,rachelle b,Trina Moore,Clyde Q,Gary094
5,Bonifacio Global City,Bradley,Elena Ciocan,travelbud.net,KGB777,luxetrip,AJ I,Makoto,inhale-exhale321,Pasifiku671,...,Jane S,Nancy de Guia,mabuhay878,LoveToTravel6467,SaadTowheed,gabayali,sauraumpfa,pinyinok,Kizomama,Cato K
6,Intramuros,Elena Ciocan,Cheska B,Marty,Marimar A,Shafiq H,travelbud.net,KGB777,Aaqarsh Aiyyar,luxetrip,...,Mauro C,elxar,Makoto,LadyG,Queenie Rph,Iya Q.,Jennifer Mendoza,inhale-exhale321,Bernadine S,TravelAdventureGurus
7,Mall of Asia Arena,Kevan R,1916somme,LKMT,Katy,Ryan A,Macky M,kumarprasad00,TGDF,Rags357,...,Travel03000659419,Arnold C,Patrick G,Dadslariosa,Peter P,Alexis V,MoolooSwampy,Karl,Lee Majors,marlyn
8,Taytay Tiangge,Casey,Vincenzo P.,yolyg2015,LarryV,Shane Williams,Tadzmahal A,Shane Williams,Tadzmahal A,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Upside Down Museum,Milind Goel,Kandy G,Saul_True,Camille,ynadqnor,Fleixer,SamirArora,Theresa C,Wonderlust,...,Gelai j,jchow123,Alvs,ynadqnor,Janelle J,jenandallthethings,Weng845,28Lizavetik,Greg C,Ativoo


In [13]:
#create dataframe that will contain the address details of the tourist sites
df_address = pd.DataFrame(columns = ['tourist site','address'],index = range(0,len(links_mm_2)+1))
df_address

,tourist site,address
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [14]:
#This section extracts the addresses of the tourist sites and store the extracted data to df_address dataframe

l = 0 

#for loop to extract the address details of each tourist site
for link in links_mm_2:
    address = []
    link2 = link.replace("{}",'') #replace {} with '' to extract the address details from the first page of each tourist site webpage
    soup2 = urllib.request.urlopen(link2)
    sleep(randint(1,5))
    bsobj2 = BeautifulSoup(soup2,'lxml')
    
    #extract the headers, which contains the name of the tourist site and append to reviewers
    for s in bsobj2.find_all('h1', {'class':'DrjyGw-P _1SRa-qNz qf3QTY0F'}):
                address.append(s.getText().split('\n')[0])
    
    #extract address
    for u in bsobj2.find_all('button',{'class':'LgQbZEQC _22upaSQN _1v-QphLm _1fKqJFvt'}):
        address.append(u.getText().split('\n')[0])
        
    #filter data and store to df_address
    for v in range(0,len(address)):
        if v <= 1:
            if address[v] == df_reviewer.iloc[l,20]:
                df_address.iloc[l,v] = ''
            else:
                df_address.iloc[l,v] = address[v]
    if l <= len(links_mm_2):
        l = l + 1

#some webpage do not contain the address of the tourist site. Instead of NaN, we replace the values with ''


In [15]:
#check content
df_address = df_address.fillna('')
df_address

,tourist site,address
0,National Museum,"Padre Burgos Ave Ermita, Manila, Luzon 2004 Ph..."
1,Greenbelt Mall,"Greenbelt Drive, Makati, Luzon 1223 Philippines"
2,Art in Island,"175 15th Ave. Cubao, Quezon City, Luzon 1109 P..."
3,Fort Santiago,"Intramuros, Manila, Luzon 1002 Philippines"
4,Resorts World Manila,"Newport Blvd Newport City, Pasay, Luzon 1309 P..."
5,Bonifacio Global City,
6,Intramuros,"Bonifacio Dr & Padre Burgos St, Manila, Luzon ..."
7,Mall of Asia Arena,
8,Taytay Tiangge,"Club Manila East Compound, Luzon 1920 Philippines"
9,Upside Down Museum,"Roxas Boulevard Boom Na Boom Grounds, CCP Comp..."


In [16]:
#create dataframe that will contain the classification of the tourist site

df_classification = pd.DataFrame(columns = ['tourist site','1','2','Classification','3'],index = range(0,len(links_mm_2)+1))
df_classification

,tourist site,1,2,Classification,3
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [17]:
#This section extracts the classification of each tourist site that we extracted from Travel Advisor.

l = 0 
for link in links_mm_2:
    
    classification = [] #will temporarily store the classification data
    link2 = link.replace("{}",'')
    soup2 = urllib.request.urlopen(link2)
    sleep(randint(1,5))
    bsobj2 = BeautifulSoup(soup2,'lxml')
    
    #extract the headers, which contains the name of the tourist site and append to reviewers
    for s in bsobj2.find_all('h1', {'class':'DrjyGw-P _1SRa-qNz qf3QTY0F'}):
                classification.append(s.getText().split('\n')[0])
            
    #extract classification details
    for t in bsobj2.find_all('div', {'class':'_28X3NMFC'},{'class':'DrjyGw-P _26S7gyB4 _2nPM5Opx'}):
                classification.append(t.getText().split('\n')[0])
            
    #filter data and store to df_classification
    for v in range(0,len(classification)):
        if v <= 4:
            df_classification.iloc[l,v] = classification[v]

    if l <= len(links_mm_2):
        l = l + 1

In [18]:
#drop unnecessary columns
df_classification = df_classification.drop(columns = ['1','2','3'])

#check content
df_classification

,tourist site,Classification
0,National Museum,Speciality Museums
1,Greenbelt Mall,Shopping Malls
2,Art in Island,Art Museums
3,Fort Santiago,Historic Sites • Parks
4,Resorts World Manila,Casinos
5,Bonifacio Global City,Points of Interest & Landmarks
6,Intramuros,Neighbourhoods • Historic Walking Areas
7,Mall of Asia Arena,Arenas & Stadiums • Shopping Malls
8,Taytay Tiangge,Flea & Street Markets
9,Upside Down Museum,Speciality Museums


In [19]:
#create dataframe for the tourist site description

df_about = pd.DataFrame(columns = ['tourist site','about'],index = range(0,len(links_mm_2)+1))
df_about

,tourist site,about
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [20]:
#This section extracts the description of the tourist sites from Travel Advisor

l = 0 
for link in links_mm_2:
    about = []
    link2 = link.replace("{}",'') #replace {} with '' to extract the data from the first page of the tourist site webpage
    soup2 = urllib.request.urlopen(link2)
    sleep(randint(1,5))
    bsobj2 = BeautifulSoup(soup2,'lxml')
    
    #extract the headers, which contains the name of the tourist site and append to reviewers
    for s in bsobj2.find_all('h1', {'class':'DrjyGw-P _1SRa-qNz qf3QTY0F'}):
                about.append(s.getText().split('\n')[0])
            
    #extract the description
    for t in bsobj2.find_all('div', {'class':'lyGV4Xx2'},{'class':'DrjyGw-P _26S7gyB4 _2nPM5Opx'}):
                about.append(t.getText().split('\n')[0])
    
    #filter data and store to df_about
    for v in range(0,len(about)):
        if v <= 1:
            df_about.iloc[l,v] = about[v]
    if l <= len(links_mm_2):
        l = l + 1

In [21]:
#replace Nan values with blank values
df_about = df_about.fillna('No Data')
df_about

,tourist site,about
0,National Museum,This grand exhibit house features the country'...
1,Greenbelt Mall,No Data
2,Art in Island,No Data
3,Fort Santiago,This museum and public park was built as a sto...
4,Resorts World Manila,No Data
5,Bonifacio Global City,No Data
6,Intramuros,"Intramuros, ""The Walled City,"" is the oldest d..."
7,Mall of Asia Arena,No Data
8,Taytay Tiangge,No Data
9,Upside Down Museum,Upside Down Museum is the newest craze in Mega...


In [22]:
#create a dataframe that will contain the entrance fee data
df_classification_price = df_classification
df_classification_price['Entrance Fee (PHP)'] = ''
df_classification_price

,tourist site,Classification,Entrance Fee (PHP)
0,National Museum,Speciality Museums,
1,Greenbelt Mall,Shopping Malls,
2,Art in Island,Art Museums,
3,Fort Santiago,Historic Sites • Parks,
4,Resorts World Manila,Casinos,
5,Bonifacio Global City,Points of Interest & Landmarks,
6,Intramuros,Neighbourhoods • Historic Walking Areas,
7,Mall of Asia Arena,Arenas & Stadiums • Shopping Malls,
8,Taytay Tiangge,Flea & Street Markets,
9,Upside Down Museum,Speciality Museums,


In [23]:
#create a dictionary that contains the entrance fee prices from other sources
classification_price_2 = {
    'National Museum' : '0', #Based on http://www.nationalmuseum.gov.ph/nationalmuseumbeta/Visit/Booking.html
    'Art in Island' : '500', #Based on https://www.facebook.com/artinisland/about/
    'Upside Down Museum' : '450', #Based on https://www.upsidedown.com.ph/about
    'San Agustin Museum' : '200', #Based on https://thequeensescape.com/a-tour-around-san-agustin-church-and-museum-in-intramuros-manila-philippines/#:~:text=in%20the%20country.-,San%20Agustin%20Museum%20Entrance%20Fee,AM%20to%206%3A00%20PM.
    'National Museum of Natural History' : '0',
    'Manila Ocean Park': '570' #Based on https://www.manilaoceanpark.com/
}

#enter the data to the Entrance Fee column of the df_classification_price dataframe
df_classification_price['Entrance Fee (PHP)'] = df_classification_price['tourist site'].map(classification_price_2)
df_classification_price

,tourist site,Classification,Entrance Fee (PHP)
0,National Museum,Speciality Museums,0
1,Greenbelt Mall,Shopping Malls,NaN
2,Art in Island,Art Museums,500
3,Fort Santiago,Historic Sites • Parks,NaN
4,Resorts World Manila,Casinos,NaN
5,Bonifacio Global City,Points of Interest & Landmarks,NaN
6,Intramuros,Neighbourhoods • Historic Walking Areas,NaN
7,Mall of Asia Arena,Arenas & Stadiums • Shopping Malls,NaN
8,Taytay Tiangge,Flea & Street Markets,NaN
9,Upside Down Museum,Speciality Museums,450


In [24]:
#Create a list containing the tourist site classifications with free entrance fee

classification_free = ['Parks','Shopping Malls','Landmarks','Historic Walking Areas','Flea & Street Markets','Bridges',
                      'Religious Sites','Neighbourhoods','Markets','Gardens','Casinos']

#Using a for loop and the classification_fee list, place zero to the entrance fee column of the the tourist sites that belong to
#the classification_free list

for i in range(0,len(classification_free)):
    for j in range(0,len(df_classification_price)-1):
        print(classification_free[i])
        print(df_classification_price.iloc[j,1])
        if classification_free[i] in df_classification_price.iloc[j,1]:
            df_classification_price.iloc[j,2] = '0'

Parks
Speciality Museums
Parks
Shopping Malls
Parks
Art Museums
Parks
Historic Sites • Parks
Parks
Casinos
Parks
Points of Interest & Landmarks
Parks
Neighbourhoods • Historic Walking Areas
Parks
Arenas & Stadiums • Shopping Malls
Parks
Flea & Street Markets
Parks
Speciality Museums
Parks
Shopping Malls
Parks
Bridges
Parks
Religious Sites
Parks
History Museums
Parks
Casinos
Parks
Amusement & Theme Parks
Parks
Water Parks
Parks
Amusement & Theme Parks
Parks
Casinos
Parks
Casinos
Parks
Shopping Malls
Parks
Neighbourhoods
Parks
Farmers Markets
Parks
Gardens
Parks
Shopping Malls
Parks
Religious Sites
Parks
Natural History Museums
Parks
Parks
Parks
Aquariums
Parks
Shopping Malls
Shopping Malls
Speciality Museums
Shopping Malls
Shopping Malls
Shopping Malls
Art Museums
Shopping Malls
Historic Sites • Parks
Shopping Malls
Casinos
Shopping Malls
Points of Interest & Landmarks
Shopping Malls
Neighbourhoods • Historic Walking Areas
Shopping Malls
Arenas & Stadiums • Shopping Malls
Shopping Malls

In [25]:
#check content
df_classification_price

,tourist site,Classification,Entrance Fee (PHP)
0,National Museum,Speciality Museums,0
1,Greenbelt Mall,Shopping Malls,0
2,Art in Island,Art Museums,500
3,Fort Santiago,Historic Sites • Parks,0
4,Resorts World Manila,Casinos,0
5,Bonifacio Global City,Points of Interest & Landmarks,0
6,Intramuros,Neighbourhoods • Historic Walking Areas,0
7,Mall of Asia Arena,Arenas & Stadiums • Shopping Malls,0
8,Taytay Tiangge,Flea & Street Markets,0
9,Upside Down Museum,Speciality Museums,450


In [26]:
#merge the five dataframes containing the attributes of the tourist sites

df_tourist_site = df_review.merge(df_reviewer, on = 'Tourist Site', how = 'left', suffixes = ['_reviews','reviewer'])
df_tourist_site = df_tourist_site.merge(df_address, how = 'left', left_on = 'Tourist Site', right_on = 'tourist site')
df_tourist_site = df_tourist_site.drop(columns = ['tourist site'])
df_tourist_site = df_tourist_site.merge(df_classification_price, how = 'left', left_on = 'Tourist Site', right_on = 'tourist site')
df_tourist_site = df_tourist_site.drop(columns = ['tourist site'])
df_tourist_site = df_tourist_site.merge(df_about, how = 'left', left_on = 'Tourist Site', right_on = 'tourist site')
df_tourist_site = df_tourist_site.drop(columns = ['tourist site'])

In [27]:
#re-order the columns of df_tourist_site

df_tourist_site = df_tourist_site[['Tourist Site','Classification','about','Entrance Fee (PHP)','address','review_1','reviewer_1',
                    'review_2','reviewer_2','review_3','reviewer_3',
                    'review_4','reviewer_4','review_5','reviewer_5','review_6','reviewer_6',
                    'review_7','reviewer_7','review_8','reviewer_8',
                    'review_9','reviewer_9','review_10','reviewer_10','review_11','reviewer_11',
                    'review_12','reviewer_12','review_13','reviewer_13',
                    'review_14','reviewer_14','review_15','reviewer_15','review_16','reviewer_16',
                     'review_17', 'reviewer_17','review_18','reviewer_18',
                    'review_19', 'reviewer_19','review_20','reviewer_20']]

In [28]:
#drop row which has NaN in Tourist Site column

df_tourist_site = df_tourist_site.dropna(subset = ['Tourist Site'])

#check content of df_tourist_site
df_tourist_site

,Tourist Site,Classification,about,Entrance Fee (PHP),address,review_1,reviewer_1,review_2,reviewer_2,review_3,...,review_16,reviewer_16,review_17,reviewer_17,review_18,reviewer_18,review_19,reviewer_19,review_20,reviewer_20
0,National Museum,Speciality Museums,This grand exhibit house features the country'...,0,"Padre Burgos Ave Ermita, Manila, Luzon 2004 Ph...",We really loved the paintings at the National ...,Hellen Mai,"Upon entering, the first major painting was th...",audreyb120,"Structure is ok but contentwise, nothing much....",...,One of the underrated repositories of art in A...,Steven T,Allot one whole day for you and your family to...,Choo-Beng G,The PI is not a rich nation and lacks the dept...,Jennifer G,This museum is located in the heart of the cou...,gea4957,Many exhibits covering different regions of Ph...,CounselorEK
1,Greenbelt Mall,Shopping Malls,No Data,0,"Greenbelt Drive, Makati, Luzon 1223 Philippines",Nice mall for good walk and shopping! You wil...,grace virlouvet,"Restaurant and coffee shops are everywhere, br...",Garland V,"This is a high end mall, not that different fr...",...,nice mall favoritr one more to offrr likr rest...,Elaine F,Great place for shopping and dining out Lots o...,Eugene,Greenbelt mall is very large with 5 malls in o...,Waleed AlWafi,Gosh this place was overwhelming at times but ...,International_Traveler,Greenbelt is a nice a place to to shop and eat...,Timetravelbug
2,Art in Island,Art Museums,No Data,500,"175 15th Ave. Cubao, Quezon City, Luzon 1109 P...",it's a one-of-a-kind showcase of art. The 3-d...,Jane V,Explore your creativity on all the pictures yo...,milo_dapilos,This was the second time I've gone to art in i...,...,A fun and memorable experience especially for ...,Cassandra Yu,This place is a lot of fun to visit with some ...,Jase,Be part of the art - take photos from prescrib...,Darren B,"Me, my mom and my kids enjoyed our visit here....",JAASM,Best time to go is when you have already eaten...,zheeby2872
3,Fort Santiago,Historic Sites • Parks,This museum and public park was built as a sto...,0,"Intramuros, Manila, Luzon 1002 Philippines",You can see the Spanish glory in the beautiful...,Elena Ciocan,"As Christians, this place boost My faith, feed...",Marimar A,Pretty park in the old city. It was nice to w...,...,"Beautiful place, so clean and many scenic area...",Bernadine S,Fort Santiago is a treasure in the middle of t...,Mambo305,Place is well maintained. I could see that the...,Alan L,This one will make you see the history of Riza...,Nico Samneil Sy Salado,"When you go to Intramuros, the old town of Man...",dongjerms
4,Resorts World Manila,Casinos,No Data,0,"Newport Blvd Newport City, Pasay, Luzon 1309 P...",Didn't expect that the casino had a small mall...,Mr Prinz,You can find luxury goods here... enjoyed the ...,Fred,We were impressed at the huge complex built he...,...,This is a highly recommended hotel especially ...,Al ali,"Enjoyed it here, very up market, great casino ...",rachelle b,When me and my friend go to resorts world mani...,Trina Moore,Located in Manila not far from Terminal 3 airp...,Clyde Q,I enjoyed playing here even if 8 didn't win an...,Gary094
5,Bonifacio Global City,Points of Interest & Landmarks,No Data,0,,"A great place to hang-out. Clean, modern and s...",Bradley,"Nothing to to with Pilipino spirit, but defini...",Elena Ciocan,"The malls and the park here is very relaxing, ...",...,this is one of the few places in the mess of M...,gabayali,It's like being in prison because it's so awfu...,sauraumpfa,We dine in Mango tree- the food are good and t...,pinyinok,"Lots of restaurants, shops, malls and open spa...",Kizomama,"BGC by the name it has, its a definite Global ...",Cato K
6,Intramuros,Neighbourhoods • Historic Walking Areas,"Intramuros, ""The Walled City,"" is the oldest d...",0,"Bonifacio Dr & Padre Burgos St, Manila, Luzon ...",This place is a reminder if the Spanish times ...,Elena Ciocan,I love seeing old man-made houses or esta

In [29]:
#create a dictionary containing the addresses of the tourist sites that have no address details in Travel Advisor and supply
#the extracted addresses from other sources

missing_address = {
    'Bonifacio Global City' : 'Bonifacio Global City, Taguig City',
    'Mall of Asia Arena' : 'SM Mall of Asia,Seaside Blvd., Pasay City, Philippines', #Based from https://www.smsupermalls.com/mall-directory/sm-mall-of-asia/information/
    'Robinsons Place Mall' : 'Pedro Gil cor. Adriatico Streets, Ermita, Manila', #Based from https://www.robinsonsmalls.com/mall-info/robinsons-place-manila
    'Eastwood City' : 'Eulogio Rodriguez Jr. Ave, Bagumbayan, Quezon City, 1800 Metro Manila', #Based from https://www.megaworldcorp.com/townships/eastwood-city
    'Ayala Triangle Gardens' : 'Paseo De Roxas St Cor Makati Ave Cor Ayala Ave, Makati, 1209 Metro Manila', #Based from https://www.makeitmakati.com/ayala-triangle-gardens/
    'Greenhills Shopping Center' : 'Ortigas Avenue, San Juan, Metro Manila, Philippines', #Based from https://www.greenhills.com.ph/about.php
    'Manila Ocean Park' : 'Manila Ocean Park, Luneta, Manila, Philippines 1000', #Based from https://www.manilaoceanpark.com/
    'Alabang Town Center' : 'Alabang Town Center, Muntinlupa City, 1780' #Based from https://www.ayalamalls.com/main/malls/ayala-alabang-town-centre-mall
}


#using for loop and missing_address, fill in the missing address details with the extracted addresses from other sources

df_tourist_site['address_2'] = df_tourist_site['Tourist Site'].map(missing_address)
size = df_tourist_site.shape
for i in range(0,len(df_tourist_site)):
    if (df_tourist_site.iloc[i,4] == ''):
        df_tourist_site.iloc[i,4] = df_tourist_site.iloc[i,size[1]-1]

df_tourist_site = df_tourist_site.drop(columns = ['address_2'])

In [30]:
#store data to tourist_sites.csv
df_tourist_site.to_csv('Tourist_site.csv')

In [31]:
df_tourist_site

,Tourist Site,Classification,about,Entrance Fee (PHP),address,review_1,reviewer_1,review_2,reviewer_2,review_3,...,review_16,reviewer_16,review_17,reviewer_17,review_18,reviewer_18,review_19,reviewer_19,review_20,reviewer_20
0,National Museum,Speciality Museums,This grand exhibit house features the country'...,0,"Padre Burgos Ave Ermita, Manila, Luzon 2004 Ph...",We really loved the paintings at the National ...,Hellen Mai,"Upon entering, the first major painting was th...",audreyb120,"Structure is ok but contentwise, nothing much....",...,One of the underrated repositories of art in A...,Steven T,Allot one whole day for you and your family to...,Choo-Beng G,The PI is not a rich nation and lacks the dept...,Jennifer G,This museum is located in the heart of the cou...,gea4957,Many exhibits covering different regions of Ph...,CounselorEK
1,Greenbelt Mall,Shopping Malls,No Data,0,"Greenbelt Drive, Makati, Luzon 1223 Philippines",Nice mall for good walk and shopping! You wil...,grace virlouvet,"Restaurant and coffee shops are everywhere, br...",Garland V,"This is a high end mall, not that different fr...",...,nice mall favoritr one more to offrr likr rest...,Elaine F,Great place for shopping and dining out Lots o...,Eugene,Greenbelt mall is very large with 5 malls in o...,Waleed AlWafi,Gosh this place was overwhelming at times but ...,International_Traveler,Greenbelt is a nice a place to to shop and eat...,Timetravelbug
2,Art in Island,Art Museums,No Data,500,"175 15th Ave. Cubao, Quezon City, Luzon 1109 P...",it's a one-of-a-kind showcase of art. The 3-d...,Jane V,Explore your creativity on all the pictures yo...,milo_dapilos,This was the second time I've gone to art in i...,...,A fun and memorable experience especially for ...,Cassandra Yu,This place is a lot of fun to visit with some ...,Jase,Be part of the art - take photos from prescrib...,Darren B,"Me, my mom and my kids enjoyed our visit here....",JAASM,Best time to go is when you have already eaten...,zheeby2872
3,Fort Santiago,Historic Sites • Parks,This museum and public park was built as a sto...,0,"Intramuros, Manila, Luzon 1002 Philippines",You can see the Spanish glory in the beautiful...,Elena Ciocan,"As Christians, this place boost My faith, feed...",Marimar A,Pretty park in the old city. It was nice to w...,...,"Beautiful place, so clean and many scenic area...",Bernadine S,Fort Santiago is a treasure in the middle of t...,Mambo305,Place is well maintained. I could see that the...,Alan L,This one will make you see the history of Riza...,Nico Samneil Sy Salado,"When you go to Intramuros, the old town of Man...",dongjerms
4,Resorts World Manila,Casinos,No Data,0,"Newport Blvd Newport City, Pasay, Luzon 1309 P...",Didn't expect that the casino had a small mall...,Mr Prinz,You can find luxury goods here... enjoyed the ...,Fred,We were impressed at the huge complex built he...,...,This is a highly recommended hotel especially ...,Al ali,"Enjoyed it here, very up market, great casino ...",rachelle b,When me and my friend go to resorts world mani...,Trina Moore,Located in Manila not far from Terminal 3 airp...,Clyde Q,I enjoyed playing here even if 8 didn't win an...,Gary094
5,Bonifacio Global City,Points of Interest & Landmarks,No Data,0,"Bonifacio Global City, Taguig City","A great place to hang-out. Clean, modern and s...",Bradley,"Nothing to to with Pilipino spirit, but defini...",Elena Ciocan,"The malls and the park here is very relaxing, ...",...,this is one of the few places in the mess of M...,gabayali,It's like being in prison because it's so awfu...,sauraumpfa,We dine in Mango tree- the food are good and t...,pinyinok,"Lots of restaurants, shops, malls and open spa...",Kizomama,"BGC by the name it has, its a definite Global ...",Cato K
6,Intramuros,Neighbourhoods • Historic Walking Areas,"Intramuros, ""The Walled City,"" is the oldest d...",0,"Bonifacio Dr & Padre Burgos St, Manila, Luzon ...",This place is a reminder if the Spanish times ...,Elena Ciocan,I lov